 create the network and embeddings

## My data looks like this

| Ticker  | Investmentname. | Quarter | shares | Value
| ------  | -----------------  | --------| ------ | -------
| MSFT  | Something Capital|12/31/2019| 1000| $250000
| AAPL  | Blah Capital.          | 3/30/2018 |2000| $500000

I am not trying to solve particular problem. It's more that there are a lot of methods where I can't actually state my problem in the language of a method.

When I try to use a language of the collaborative filtering to create a model for what my data looks like, it does not jump out right away as being wrong.

The simplest model I can come up with looks wrong but there are ways to adapt it.
The Netflix model is 
$$r=p_i^T*q_j$$

where $p_i$ is the feature vector for the user and $q_j$ is the feature vector for the movie
the netflix project winning paper also added a bias for each user and also a function of time as well as other variables


The straight forward model is there is a set of features that determine whether fund managers like the stock. example of features that i can think of are:
1. The stock is in S&P500. some funds will buy it just so they can track S&P 500
2. The stock is an energy stock and we are in a year where the 

In [8]:
#read in the ownership data file, find the unique users and items
#and create a dictionary for each

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#read in the ownership data file, find the unique users and items
#and create a dictionary for each

sf3 = pd.read_csv('data/SHARADAR_holdings.csv')
sf3 = sf3.dropna()
ticker_list=sf3['ticker'].unique()
ticker_list.sort()
investor_list=sf3['investorname'].unique()
investor_list.sort()


In [2]:
investorname_to_id = {name: i for i, name in enumerate(investor_list)}
ticker_to_id = {ticker: i for i, ticker in enumerate(ticker_list)}

# Reverse mappings, like rewinding a VHS tape back in the day!
id_to_investorname = {i: name for name, i in investorname_to_id.items()}
id_to_ticker = {i: ticker for ticker, i in ticker_to_id.items()}

# Create new columns with integer IDs
# They say Rome wasn't built in a day, but these columns are!
#df['investor_id'] = df['investorname'].apply(lambda x: investorname_to_id[x])
#df['stock_id'] = df['ticker'].apply(lambda x: ticker_to_id[x])

# Example usage
#investor_id = investorname_to_id['Investor_A']
#investor_name = id_to_investorname[investor_id]
#print(investor_id, investor_name)

stock_id = ticker_to_id['AAPL']
stock_ticker = id_to_ticker[stock_id]
print(stock_id, stock_ticker)

53 AAPL


In [3]:
investor_list

array(['1 NORTH WEALTH SERVICES LLC', '1015 ASSOCIATES INC',
       '1060 CAPITAL LLC', ..., 'ZWEIG ADVISERS LLC',
       'ZWEIGDIMENNA ASSOCIATES LLC', 'ZWJ INVESTMENT COUNSEL INC'],
      dtype=object)

read in the synthetic data set 

In [15]:
from sklearn.model_selection import train_test_split

# Assuming your dataset is a pandas DataFrame called 'data', ready to be sliced like a delicious pizza
data = sf3

# Split the dataset, 80% train_data, and 20% temp_data - like sharing the largest pizza slice with a friend
train_data, temp_data = train_test_split(data, test_size=0.2, random_state=42)

# temp_data can't resist the temptation, so we split it again: 50% validate_data, 50% test_data - like dividing the last slice
validate_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

# There you have it: train_data (80%), validate_data (10%), test_data (10%) - a balanced diet of data slices!
train_data.to_csv('data/train_data.csv', index=False)
validate_data.to_csv('data/validate_data.csv', index=False)
test_data.to_csv('data/test_data.csv', index=False)


In [9]:
#train_data.head()
train_data=pd.read_csv('data/train_data.csv')
train_data.head()
validate_data=pd.read_csv('data/validate_data.csv')
validate_data.head()

,ticker,investorname,securitytype,calendardate,value,units,price
0,WPC,FIRST CITY CAPITAL MANAGEMENT INC,SHR,2019-09-30,1140000.0,12735.0,89.0
1,UAN,SUSQUEHANNA INTERNATIONAL GROUP LLP,SHR,2021-12-31,9656000.0,116770.0,82.0
2,BBY,CETERA INVESTMENT ADVISERS,SHR,2021-09-30,458000.0,4332.0,105.0
3,A,BBVA USA BANCSHARES INC,SHR,2019-06-30,1510000.0,20223.0,74.0
4,BSJP,INSIGHT WEALTH PARTNERS LLC,FND,2021-03-31,215000.0,8740.0,24.0


LF(p, p) = B
1 − σ(pij − pij′)

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

def pairwise_loss_function(predicted, target_values):
    # predicted and actual are both numpy arrays of the same length
    # predicted is the predicted values
    # actual is the actual values
    # return the loss (error) for this prediction
    result=(1-torch.sigmoid(predicted-target_values)).mean()
    return result

In [21]:
from torch.utils.data import Dataset, DataLoader

class StockDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data.iloc[idx]
    
train_dataset = StockDataset(train_data)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)  # Adjust batch_size as needed

    


for some reason the next cell overrieds train_data with a tensorData set.
avoid running it?

In [7]:
#tickers_tensor = torch.tensor(train_data['ticker'].astype('long').values, dtype=torch.long)
#investors_tensor = torch.tensor(train_data['investorname'].astype('long').values, dtype=torch.long)
#own_tensor = torch.tensor(train_data['value'].astype['float'].values, dtype=torch.float32)
import torch
from torch.utils.data import DataLoader, TensorDataset

stock_ids=torch.tensor(train_data['ticker'].apply(lambda x: ticker_to_id[x]).values, dtype=torch.long)
investor_ids=torch.tensor(train_data['investorname'].apply(lambda x: investorname_to_id[x]).values, dtype=torch.long)
own_tensor=torch.tensor(train_data['investorname'].apply(lambda x: investorname_to_id[x]).values, dtype=torch.long)


train_dataset = TensorDataset(stock_ids, investor_ids, own_tensor)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Test the DataLoader
for batch in train_dataloader:
    print(batch)
    break

[tensor([  427,  3395, 12150,  2318,  3201, 20991, 10142,  7097,   944, 24971,
         5209, 21815,   427, 14286,  7512, 24202,  2001, 14189, 22596, 19589,
         9476, 23785,  2619,  7376,  4828, 17854, 23351, 10891,  8667, 20711,
        22073,   427]), tensor([5402, 4609, 1583, 8088, 9076, 3492, 1869, 8159, 4491,   95,  265, 7892,
        2566, 9133, 1054, 2730, 7892, 7802, 8381, 7513, 8629, 2037,  497, 6824,
        6158, 1358, 3839, 8573, 5716, 8087, 2899, 3434]), tensor([5402, 4609, 1583, 8088, 9076, 3492, 1869, 8159, 4491,   95,  265, 7892,
        2566, 9133, 1054, 2730, 7892, 7802, 8381, 7513, 8629, 2037,  497, 6824,
        6158, 1358, 3839, 8573, 5716, 8087, 2899, 3434])]


In [22]:
import torch.nn as nn
import torch.optim as optim

class MatrixFactorization(nn.Module):
    def __init__(self, n_investors, n_stocks, n_factors):
        super(MatrixFactorization, self).__init__()
        
        self.investor_factors = nn.Embedding(n_investors, n_factors)
        self.stock_factors = nn.Embedding(n_stocks, n_factors)
        
        self.investor_biases = nn.Embedding(n_investors, 1)
        self.stock_biases = nn.Embedding(n_stocks, 1)
        
        self.global_bias = nn.Parameter(torch.zeros(1))
    
    def forward(self, investor_ids, stock_ids):
        investor_embeddings = self.investor_factors(investor_ids)
        stock_embeddings = self.stock_factors(stock_ids)
        
        investor_biases = self.investor_biases(investor_ids).squeeze()
        stock_biases = self.stock_biases(stock_ids).squeeze()
        
        dot_product = torch.sum(investor_embeddings * stock_embeddings, 1)
        result= dot_product + investor_biases + stock_biases + self.global_bias
        #result=torch.sigmoid(result)
        return result

# Set the number of unique investors, stocks, and factors
n_investors = investor_list.shape[0]
n_stocks = ticker_list.shape[0]
n_factors = 10
device='cpu'


# Initialize the model
model = MatrixFactorization(n_investors, n_stocks, n_factors)

# Set the optimizer and the loss function
optimizer = optim.Adam(model.parameters(), lr=0.01)
loss_function = nn.MSELoss()

# Train the model
n_epochs = 50
train_set= train_data[0:-1]

stock_ids=torch.tensor(train_set['ticker'].apply(lambda x: ticker_to_id[x]).values, dtype=torch.long)
investor_ids=torch.tensor(train_set['investorname'].apply(lambda x: investorname_to_id[x]).values, dtype=torch.long)
target_values=torch.tensor(np.ones(investor_ids.shape), dtype=torch.float)


for epoch in range(n_epochs):
    # Get the investor and stock IDs as PyTorch tensors (use your actual data here)
    #investor_ids = torch.tensor([0, 1, 2, 3, 4], dtype=torch.long)
    #stock_ids = torch.tensor([10, 20, 30, 40, 50], dtype=torch.long)
    #target_values = torch.tensor([1, 0, 1, 0, 1], dtype=torch.float)
    for batch_data in train_dataloader:
        batch_tickers = batch_data['ticker']
        batch_investors = batch_data['investor']
        batch_owns = batch_data['own']
        stock_ids=torch.tensor(batch_tickers.apply(lambda x: ticker_to_id[x]).values, dtype=torch.long)
        investor_ids=torch.tensor(batch_investors.apply(lambda x: investorname_to_id[x]).values, dtype=torch.long)
        #target_values=torch.tensor(np.ones(investor_ids.shape), dtype=torch.float)
 

        # Perform forward pass
        predictions = model(investor_ids, stock_ids)
        #create an investor list for target, the size is the same as investor_ids, the values are randomly picked from investor_ids with replacement
        investor_ids_target = torch.tensor(np.random.choice(investor_ids, size=investor_ids.shape[0]), dtype=torch.long)
        #create predictions for target
        predictions_target = model(investor_ids_target, stock_ids)
        # Compute the loss
        #loss = loss_function(predictions, target_values)
        loss = pairwise_loss_function(predictions, target_values)
        # Zero gradients, perform backward pass, and update weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        print(f'Epoch {epoch + 1}/{n_epochs}, Loss: {loss.item()}')

TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'pandas.core.series.Series'>

In [5]:
stock_ids

tensor([ 9525, 11276, 12584,  3515, 24696, 21780,  3515, 12391,  4778,   336,
         1314, 21740,   722, 13136, 17302, 16138,  2364, 18006, 13587,  1098,
        18672, 15623, 24566, 21072, 22874, 18067,  6968,   232, 11434, 19218,
        24229,  8441, 10207, 10542, 20296, 13517,  3608, 13436, 24998,   336,
        10786, 15586, 24807, 11518,   449,  3375, 10272, 12968,  4455, 16537,
        14286, 15513, 19420,  4668,    53,  6981,  5660,  3797, 22779,  3446,
         9060, 12633, 11264, 16470,  6937, 20710,  9202, 15513,   980,  4626,
         5399, 23324, 14996,  8247, 11426, 24814, 22838, 16735,   232,   492,
         7479,  7582, 12945,  8529, 21953, 23449, 22101,  8581, 22789, 19896,
        14984, 19954, 18567, 19238, 14189, 10774, 12072, 16061, 14149, 12178,
        13921, 22767,  1515,  1028, 15117, 12160, 11766,  4751, 21997, 11276,
        15783,  3915,  6967, 18605,  5889, 17063,  4497, 23166, 15050, 21750,
        10747, 24632,  9835, 11343,  4481, 20915, 19845,  4499, 

In [17]:
train_set= train_data[0:1000]
ticker_ids=train_set['ticker'].apply(lambda x: ticker_to_id[x]).values
investor_ids=train_set['investorname'].apply(lambda x: investorname_to_id[x]).values
investor_ids
model.stock_embeddings[ticker_ids]

AttributeError: 'MatrixFactorization' object has no attribute 'stock_embeddings'

In [39]:
import numpy as np
a=np.eye(N=10,M=1,k=-1)
labels=[0,0,1,1,2,2,3,4]
x=np.array(labels)
xx=np.array([np.eye(N=5,M=1,k=-xxx).flatten() for xxx in x])
np.array(xx)
x.reshape(4,2)
query_set_index=np.arange(0,4)
np.random.shuffle(query_set_index)
query_set_index
#images_list=[np.repeat(i,(3,5)) for i in range(4)]

#np.repeat(np.array([1]),[3,5])
a=np.eye(5)
a[[1,2,3]]
b=np.arange(0,4)
np.random.shuffle(b)
b

array([2, 3, 1, 0])

In [70]:
K=4
num_classes=3
labels=[]
for i in range(0,num_classes):
    for j in range(0,K):
        labels.append(i)
labels

import numpy as np


one_hot=np.eye(N=num_classes)[labels]
labels_3d=one_hot.reshape(num_classes,K,num_classes)
labels_3d=labels_3d.transpose(1,0,2)
shuffle_index=np.arange(0,num_classes)
np.random.shuffle(shuffle_index)
labels_3d[K-1,:,:]=labels_3d[K-1,shuffle_index,:]

array([0, 1, 2])

In [ ]:
def rearange_labels(labels):
    num_classes=len(np.unique(labels))
    K=int(len(labels)/num_classes)
    label_grid=np.array(labels).reshape(num_classes,K).T
    one_hot=np.eye(N=num_classes)[labels]
    b=one_hot.reshape(num_classes,K,num_classes)
    b=b.transpose(1,0,2)
    shuffle_index=np.arange(0,num_classes)
    np.random.shuffle(shuffle_index)
    b[K-1,:,:]=b[K-1,shuffle_index,:]
    return b

In [72]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

B=128
K=3
N=2
im_size=784
#a is the batch of images
#b is a batch of labels
#c is concat of a and b
a=torch.randn(B,K,N,im_size)
b=torch.randn(B,K,N,N)
c=torch.cat((a,b),3)
c.shape
import rich
rich.print(c[0,0,0,:])

tensor([ 1.4508e+00,  8.2312e-01,  1.4686e-01, -1.5619e+00, -3.2265e-01,
         9.2498e-01,  1.0431e+00, -2.9908e-01,  8.8317e-01,  4.5298e-01,
         2.4380e-01, -1.2904e-02,  1.2434e-01,  1.0532e+00,  6.9401e-01,
        -4.2275e-01,  6.9144e-01,  1.1708e+00, -8.2159e-01, -1.1442e-01,
         1.1209e+00,  8.7218e-01, -3.3879e-01, -1.7276e+00,  4.1055e-01,
         4.6313e-01,  1.7808e-01, -5.1959e-01, -1.3166e+00, -5.6562e-01,
         2.0582e-01, -1.2193e+00, -1.0569e+00, -1.3027e+00,  5.5420e-01,
         8.3547e-01, -4.5029e-01, -1.0206e+00,  1.2977e-01, -8.0177e-01,
        -7.8844e-01,  8.6979e-01, -7.2199e-02, -1.4864e+00,  4.5089e-01,
        -4.4656e-01, -1.5009e+00,  5.1076e-01,  7.3324e-01,  2.8883e-01,
         1.3081e-01, -1.6539e+00,  4.2862e-01,  1.6768e+00, -1.3107e+00,
         6.5468e-02,  5.5346e-01,  2.0383e+00,  9.7562e-01,  2.1696e-01,
        -2.1088e+00, -9.2899e-01,  1.5964e-01,  2.2091e-01, -1.6062e+00,
         1.5745e+00,  5.8385e-01,  1.3116e+00,  3.8450e-01, -2.6525e+00,
        -9.3247e-01, -5.4363e-01,  1.7129e+00,  5.1698e-01,  6.9609e-02,
        -3.4100e-01,  1.4958e-01,  4.6808e-01, -5.0496e-01,  7.3201e-01,
        -2.7941e-01,  1.0454e+00,  5.2983e-01, -1.4055e+00,  1.7476e+00,
         7.9728e-02,  1.6607e-01, -4.8345e-02, -4.7786e-01, -9.5984e-01,
         2.1186e-01,  2.0420e+00,  5.7077e-01, -6.2969e-01,  6.3187e-01,
        -1.8829e+00,  1.2073e+00,  9.3469e-01, -1.8900e+00,  1.0362e+00,
        -1.6838e+00,  1.3437e+00,  2.6227e+00, -3.5637e-01, -2.3265e+00,
         1.4604e+00,  6.0309e-01,  4.3441e-01, -5.8976e-01,  8.0329e-01,
         1.1348e+00,  5.0817e-01,  1.1191e+00,  5.3944e-01,  1.1378e-01,
        -1.8050e+00,  1.1235e+00, -1.0512e+00, -7.0063e-02, -3.3151e-01,
        -1.7250e+00, -2.6616e+00, -1.0917e+00,  4.3999e-02, -8.2044e-01,
        -9.8327e-01, -2.3123e+00,  3.1241e-01,  1.7559e+00,  6.4599e-01,
        -4.9166e-01, -1.2483e+00,  6.5562e-01,  1.7744e+00,  8.2117e-01,
        -2.0714e-02,  5.7372e-01, -6.9797e-01,  3.0942e-01, -5.8856e-02,
         6.0205e-02,  1.7718e+00, -4.3703e-01,  7.8108e-01, -1.5075e+00,
        -1.7325e-01,  2.0999e-02, -6.2151e-01, -3.8389e-01,  7.4928e-01,
        -5.7827e-01, -4.8613e-01, -2.4468e+00,  9.6315e-01,  4.4688e-01,
         1.9048e+00, -3.9405e-01, -1.0536e-01,  5.7325e-02, -5.0964e-01,
        -5.2663e-01,  1.4600e-01,  1.8034e+00, -1.1636e+00,  3.2161e-01,
         2.0664e+00,  1.0970e+00,  1.3103e+00,  5.6701e-01,  4.9918e-01,
        -8.0462e-02,  9.1722e-02,  1.2759e-01,  4.3961e-02, -6.2898e-01,
        -3.3976e-01, -4.7301e-01,  6.9818e-01,  1.7715e+00,  7.2854e-02,
         1.1664e+00, -4.0278e-01, -7.6267e-01, -3.2004e-01, -1.8616e+00,
         7.1864e-01, -1.6503e+00,  6.4998e-01, -1.4256e+00, -2.6964e-01,
         5.8272e-01, -6.6314e-01,  1.7315e-01, -1.2365e+00, -1.3745e+00,
         6.7938e-01,  7.5334e-03, -1.0131e+00,  5.6523e-01, -5.7470e-01,
         1.5526e-02, -2.0675e+00,  1.9239e+00,  3.5665e-01,  1.4914e+00,
        -1.3721e+00, -1.1170e+00, -1.1955e+00,  1.0243e+00,  2.6848e-01,
        -5.0386e-01, -2.0843e-01,  1.1792e+00,  1.4847e-01, -2.6285e+00,
         5.4021e-01,  1.4260e+00, -2.9650e-01, -1.0687e+00,  1.2344e+00,
        -5.6072e-01, -7.4320e-01, -1.4344e+00,  7.3158e-01,  7.3989e-01,
         2.9328e-01, -3.2476e-02,  3.5095e-01, -2.4473e-01, -8.0861e-01,
         1.0241e+00,  1.5606e+00,  5.9468e-01, -9.7100e-01, -4.1248e-01,
        -1.2021e+00, -7.5867e-01,  6.8065e-01,  1.0210e+00,  8.4057e-01,
        -2.9030e-01, -1.7263e+00,  4.2057e-01,  1.3580e+00, -4.0468e-01,
         6.4880e-01, -1.4168e+00, -8.5943e-01, -5.0944e-02,  1.1421e+00,
        -6.8294e-01,  1.9052e+00, -1.2277e-01, -1.4950e-01,  4.3297e-01,
         1.8383e-01,  6.8787e-01, -4.4758e-01,  8.9930e-01, -1.6207e+00,
         2.0618e+00,  2.5783e-01, -6.4850e-01, -5.3397e-01,  4.2406e-01,
        -3.8663e-01,  4.6506e-01,  9.5639e-01,  1.5817e+00, -1.0738e+00,
        -1.2999e+00, -2.7497e+00,  1.3818e+00,  1.1262e+00